In [1]:
from __future__ import division, print_function, absolute_import

import time
import random
import glob
import os.path
import textwrap

from IPython.display import display, HTML, Image, Latex

%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['figure.figsize'] = 8, 8
import seaborn as sns
sns.set_style('white')

import numpy as np
import pandas as pd
import scipy.optimize
import scipy.spatial
import scipy.stats
import cv2
from tf import transformations

Load all the data

In [130]:
# is_checkerboard = True
is_checkerboard = False
# data = np.load("../data/board_data.npz")
data = np.load("../data/marker_data.npz")

bag_files = data['bag_files']

calibration_object_marker_counts = data['calibration_object_marker_counts']
camera_marker_counts = data['camera_marker_counts']

camera_matrix = data['camera_matrix']
distortion_coeffs = data['distortion_coeffs']

T_c_o = data['cam_to_optical_frame']
camera_rb_poses = data['camera_rb_poses']

all_image_coordinates = data['image_coordinates']
all_object_coordinates = data['object_coordinates']

Start byremoving any rows where there are not enough camera or calibration markers

In [131]:
def create_insufficient_markers_mask(camera_marker_counts, calibration_object_marker_counts, expected_camera_marker_count, expected_calibration_object_marker_count):
    camera_mask = camera_marker_counts == expected_camera_marker_count
    object_mask = calibration_object_marker_counts == expected_calibration_object_marker_count
    return np.logical_and(camera_mask, object_mask)

if is_checkerboard:
    filter_mask = create_insufficient_markers_mask(
    camera_marker_counts, calibration_object_marker_counts, 5, 5)
else:
    print(calibration_object_marker_counts.shape)
    filter_mask = create_insufficient_markers_mask(
        camera_marker_counts, calibration_object_marker_counts, 6, 14)

camera_rb_poses = camera_rb_poses[filter_mask]
all_image_coordinates = all_image_coordinates[filter_mask]
all_object_coordinates = all_object_coordinates[filter_mask]

(41,)


Generate optical frame priors using rigid body poses and camera->optical frame transform (assume misalignment is 0)

In [132]:
def calculate_optical_frame_poses(camera_rb_poses, T_r_c, T_c_o):
    optical_frame_poses = []
    for camera_rigid_body_pose in camera_rb_poses:
        T_m_c = np.matmul(camera_rigid_body_pose, T_r_c)
        T_m_o = np.matmul(T_m_c, T_c_o)
        optical_frame_poses.append(T_m_o)
    return optical_frame_poses

optical_frame_priors = calculate_optical_frame_poses(camera_rb_poses, np.identity(4), T_c_o)

Find rough 2D-3D correspondences. We will improve this later.

In [133]:
def match_checkerboard_corners(image_coords, object_coords, camera_pose_prior, camera_matrix, distortion_coeffs):
    """Utilizes the fact that the image coords and object coords are in the same order bar missing corners"""
    # If they are the same length then they are in the right order
    if len(image_coords) == len(object_coords):
        return object_coords

    # This is the worst case for how wrong the indices are
    diff = len(object_coords) - len(image_coords)  
    
    _, _, rotation, translation, _ = transformations.decompose_matrix(np.linalg.inv(camera_pose_prior))
    rotation = transformations.euler_matrix(*rotation)
    rotation, _ = cv2.Rodrigues(rotation[:3, :3])
    projected_coords, _ = cv2.projectPoints(object_coords, rotation, translation, camera_matrix, distortion_coeffs)
    projected_coords = projected_coords.squeeze()
    
    matched_object_coords = []
    matched_proj_coords = []
    for i, image_coord in enumerate(image_coords):
        best_match = projected_coords[i]
        best_object_coord = object_coords[i]
        # Find the object coord that matches the image coord within the possible options
        for proj_coord, object_coord in zip(projected_coords[i:i+diff+1], object_coords[i:i+diff+1]):
            if np.linalg.norm(image_coord - proj_coord) < np.linalg.norm(image_coord - best_match):
                best_match = proj_coord
                best_object_coord = object_coord
        matched_proj_coords.append(best_match)
        matched_object_coords.append(best_object_coord)    
    return np.array(matched_object_coords).squeeze()
    

def match_markers(image_coords, object_coords, camera_pose_prior, camera_matrix, distortion_coeffs):
    """Return image_coords sorted into the same order as projected_coords."""
    _, _, rotation, translation, _ = transformations.decompose_matrix(np.linalg.inv(camera_pose_prior))
    rotation = transformations.euler_matrix(*rotation)
    rotation, _ = cv2.Rodrigues(rotation[:3, :3])
    projected_coords, _ = cv2.projectPoints(object_coords, rotation, translation, camera_matrix, distortion_coeffs)
    projected_coords = projected_coords.squeeze()
    tree = scipy.spatial.cKDTree(projected_coords)
    dists, indices = tree.query(image_coords, k=1)
    return object_coords[indices]

matched_object_coordinates = []
for i, (image_coordinates, object_coordinates, optical_frame_prior) in enumerate(zip(all_image_coordinates, all_object_coordinates, optical_frame_priors)):
    if is_checkerboard:
        matched_object_coordinates.append(match_checkerboard_corners(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))
    else:
        matched_object_coordinates.append(match_markers(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))

matched_object_coordinates = np.array(matched_object_coordinates)

Find the misalignment transforms $T_{rc}$

In [134]:
def estimate_pose_ransac(image_coords, object_coords, camera_matrix, distortion_coeffs):
    _, rotation, translation, _ = cv2.solvePnPRansac(object_coords, image_coords, camera_matrix, distortion_coeffs, flags=cv2.SOLVEPNP_P3P, iterationsCount=500)
    rotation, _ = cv2.Rodrigues(rotation)
    T_o_m = transformations.compose_matrix(translate=translation.squeeze(), angles=transformations.euler_from_matrix(rotation))
    T_m_o = np.linalg.inv(T_o_m)
    return T_m_o


calibration_transforms = []
for image_coordinates, object_coordinates, camera_rb_pose in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
    optical_frame_pose = estimate_pose_ransac(image_coordinates, object_coordinates, camera_matrix, distortion_coeffs)
    camera_link_pose = np.matmul(optical_frame_pose, np.linalg.inv(T_c_o))
    calibration_transform = np.matmul(np.linalg.inv(camera_rb_pose), camera_link_pose)
    calibration_transforms.append(calibration_transform)

Convert into (x, y, z, rx, ry, rz) format

In [135]:
def Ts_to_tfs(Ts):
    calibration_tfs = []
    for transform in calibration_transforms:
        _, _, rotation, translation, _ = transformations.decompose_matrix(transform)
        calibration_tfs.append(np.append(translation, np.degrees(rotation)))
    return calibration_tfs

calibration_tfs = Ts_to_tfs(calibration_transforms)

Remove outliers

In [136]:
def remove_outliers(calibration_tfs):
    estimate = np.array([0, 0, 0, 0, 0, 0])
    tolerences = [0.05, 0.05, 0.05, 5, 5, 5]
    estimate_mask = np.any(np.abs(calibration_tfs - estimate) < tolerences, axis=1)

    tolerences = [0.05, 0.05, 0.05, 5, 5, 5]
    medians = np.median(calibration_tfs, axis=0)
    median_mask = np.any(np.abs(calibration_tfs - medians) < tolerences, axis=1)
    
    mask = estimate_mask & median_mask
    return np.array(calibration_tfs)[mask], mask

print(len(calibration_tfs))
calibration_tf_inliers, _ = remove_outliers(calibration_tfs)
print(len(calibration_tf_inliers))

41
41


In [137]:
initial_estimate = np.mean(calibration_tf_inliers, axis=0)
print(initial_estimate)
initial_estimate = transformations.compose_matrix(translate=initial_estimate[:3], angles=np.radians(initial_estimate[3:]))

[ 0.00331062  0.01202471 -0.0108471  -0.80338718  1.26910746 -1.59938756]


Now we can use this to do better matching

In [138]:
optical_frame_priors = calculate_optical_frame_poses(camera_rb_poses, initial_estimate, T_c_o)
matched_object_coordinates = []
for i, (image_coordinates, object_coordinates, optical_frame_prior) in enumerate(zip(all_image_coordinates, all_object_coordinates, optical_frame_priors)):
    matched_object_coordinates.append(match_markers(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))
matched_object_coordinates = np.array(matched_object_coordinates)

Now recaulate the poses using all the markers

In [139]:
def estimate_pose(image_coords, object_coords, camera_matrix, distortion_coeffs, initial_guess):
    _, rotation, translation = cv2.solvePnP(object_coords, image_coords, camera_matrix, distortion_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    rotation, _ = cv2.Rodrigues(rotation)
    T_o_m = transformations.compose_matrix(translate=translation.squeeze(), angles=transformations.euler_from_matrix(rotation))
    T_m_o = np.linalg.inv(T_o_m)
    return T_m_o

better_calibration_transforms = []
for image_coordinates, object_coordinates, camera_rb_pose in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
    optical_frame_pose = estimate_pose(image_coordinates, object_coordinates, camera_matrix, distortion_coeffs, initial_estimate)
    camera_link_pose = np.matmul(optical_frame_pose, np.linalg.inv(T_c_o))
    calibration_transform = np.matmul(np.linalg.inv(camera_rb_pose), camera_link_pose)
    better_calibration_transforms.append(calibration_transform)
    
better_calibration_tfs = Ts_to_tfs(better_calibration_transforms)
print(len(calibration_tfs))
better_calibration_tfs, mask = remove_outliers(better_calibration_tfs)
print(len(calibration_tfs))

41
41


In [140]:
better_calibration = np.mean(better_calibration_tfs, axis=0)
print(better_calibration)
better_calibration = transformations.compose_matrix(translate=better_calibration[:3], angles=np.radians(better_calibration[3:]))

[ 0.00331062  0.01202471 -0.0108471  -0.80338718  1.26910746 -1.59938756]


Now optimise

In [141]:
def apply_T(T, points):
    """Convert an array of 3D points into homogeneous coords, left-multiply by T, then convert back."""
    flipped = False
    if points.shape[0] != 3:
        assert points.shape[1] == 3, "Points must be 3xN or Nx3"
        points = points.T
        flipped = True
    points_h = np.vstack((points, np.ones_like(points[0,:])))
    points_transformed_h = np.dot(T, points_h)
    points_transformed = points_transformed_h[:-1]
    if flipped:
        return points_transformed.T
    return points_transformed

# Filter outlier frames
all_image_coordinates = all_image_coordinates[mask]
matched_object_coordinates = matched_object_coordinates[mask]
camera_rb_poses = camera_rb_poses[mask]

# Project object points into camera rigid body frame beforehand, as one big array (not separated by bags)
T_RO = np.dot(better_calibration, np.linalg.inv(T_c_o))
object_coords_R = []

for image_coords, object_coords_M_sorted, T_MRs in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
    for i in range(len(image_coords)):
        object_coords_R.append(apply_T(np.linalg.inv(T_MRs), np.array([object_coords_M_sorted[i]])))
object_coords_R = np.vstack(object_coords_R)

In [142]:
# Helper functions
def se3_to_T(rvec, tvec):
    """Convert an se(3) OpenCV pose to a 4x4 transformation matrix.
    
    OpenCV poses are an Euler-Rodrigues rotation vector and a translation vector.
    """
    R, _ = cv2.Rodrigues(rvec)
    euler_angles = transformations.euler_from_matrix(R)
    return transformations.compose_matrix(angles=euler_angles, translate=np.squeeze(tvec))

def T_to_se3(T):
    """Convert a 4x4 transformation matrix to an se(3) OpenCV pose."""
    rvec, _ = cv2.Rodrigues(T[:3, :3])
    tvec = T[:3, 3]
    return rvec, tvec

def tf_to_T(tf):
    """Convert a (tx, ty, tz, rx, ry, rz) tuple to a 4x4 transformation matrix."""
    if len(tf) == 6:
        # Single transform
        return transformations.compose_matrix(angles=tf[3:], translate=tf[:3])
    else:
        # Array of transforms
        assert tf.shape[1] == 6, "Transform array must be Nx6"
        return np.vstack([tf_to_T(row) for row in tf])

def T_to_tf(T):
    """Convert a 4x4 transformation matrix to a (tx, ty, tz, rx, ry, rz) tuple."""
    if T.shape == (4, 4):
        # Single transform
        _, _, rot, trans, _ = transformations.decompose_matrix(T)
        return np.hstack((trans, rot))
    else:
        # Array of transforms
        assert T.shape[1:] == (4, 4), "Transform array must be Nx4x4"
        return np.array([T_to_tf(row) for row in T])

def project_markers(T_object_camera, markers, camera_matrix, distortion_coeffs):
    """"""
    T_camera_object = np.linalg.inv(T_object_camera)
    rvec, tvec = T_to_se3(T_camera_object)
    markers2d, _ = cv2.projectPoints(markers, rvec, tvec, camera_matrix, distortion_coeffs)
    return np.squeeze(markers2d)
    
def reprojection_error_vector(T, markers, image_coords, camera_matrix, distortion_coeffs, debug=False):
    """Return the reprojection error for each marker as a vector."""
    projected_coords = project_markers(T, markers, camera_matrix, distortion_coeffs)
    difference = projected_coords - image_coords
    squared_distance = np.linalg.norm(difference, axis=1)
    if debug:
        print(np.hstack((projected_coords, image_coords)))
        print(np.hstack((projected_coords, match_coords(projected_coords, image_coords))))
        print(difference)
    return squared_distance

def reprojection_error_mean(*args, **kwargs):
    """Return the mean reprojection error."""
    return np.mean(reprojection_error_vector(*args, **kwargs))

image_coords = np.vstack(np.vstack(all_image_coordinates))
prior = T_to_tf(better_calibration)

def f(tf_RC):
    T_RO = np.dot(tf_to_T(tf_RC), T_c_o)
    return reprojection_error_vector(T_RO, object_coords_R, image_coords, camera_matrix, distortion_coeffs)

res = scipy.optimize.least_squares(f, prior)
print(res)

 active_mask: array([ 0.,  0.,  0.,  0.,  0.,  0.])
        cost: 487.77217592548402
         fun: array([ 1.23468121,  1.07373432,  0.48632884,  1.71481866,  1.26318757,
        0.92292414,  1.66916606,  1.35331815,  1.01317364,  1.20722589,
        0.92806848,  0.45095962,  2.30686085,  0.93323885,  0.48729177,
        0.64859574,  0.9828303 ,  2.38851781,  1.67193027,  1.0720776 ,
        2.06854487,  0.36941854,  1.38017642,  0.75950071,  0.46479886,
        1.02195793,  2.44848062,  0.4930161 ,  0.68128762,  0.6725786 ,
        0.86073946,  2.39976951,  1.8260771 ,  1.30986646,  2.07980444,
        0.3124947 ,  1.78473283,  0.73460698,  0.18967941,  1.18742583,
        1.91870289,  0.47551016,  0.74088022,  0.60904144,  0.56174086,
        2.29032292,  1.52397613,  0.90770781,  1.84756669,  0.81129374,
        1.58955681,  0.81502361,  0.76282893,  0.68277279,  2.10099938,
        0.57110586,  0.60093765,  0.33599409,  0.97662595,  2.50105816,
        1.78904038,  1.44715144,  2.1

In [143]:
print(prior)
print(res.x)

print(np.mean(f(prior)))
print(np.mean(f(res.x)))

[ 0.00331062  0.01202471 -0.0108471  -0.01402175  0.0221501  -0.02791458]
[ 0.00253584  0.01420037 -0.00727408 -0.01395151  0.02482445 -0.02949012]
1.4731196427
1.16774029744


## Do the same but K-fold

In [84]:
is_checkerboard = True
# is_checkerboard = False
data = np.load("../data/board_data.npz")
# data = np.load("../data/marker_data.npz")

bag_files = data['bag_files']

calibration_object_marker_counts = data['calibration_object_marker_counts']
camera_marker_counts = data['camera_marker_counts']

camera_matrix = data['camera_matrix']
distortion_coeffs = data['distortion_coeffs']

T_c_o = data['cam_to_optical_frame']
camera_rb_poses = data['camera_rb_poses']

all_image_coordinates = data['image_coordinates']
all_object_coordinates = data['object_coordinates']


def calibrate(all_image_coordinates, all_object_coordinates, T_c_o, camera_rb_poses, camera_matrix, distortion_coeffs, is_checkerboard):
    if is_checkerboard:
        filter_mask = create_insufficient_markers_mask(
        camera_marker_counts, calibration_object_marker_counts, 5, 5)
    else:
        filter_mask = create_insufficient_markers_mask(
            camera_marker_counts, calibration_object_marker_counts, 6, 14)

    camera_rb_poses = camera_rb_poses[filter_mask]
    all_image_coordinates = all_image_coordinates[filter_mask]
    all_object_coordinates = all_object_coordinates[filter_mask]
    
    optical_frame_priors = calculate_optical_frame_poses(camera_rb_poses, np.identity(4), T_c_o)
    
    matched_object_coordinates = []
    
    for i, (image_coordinates, object_coordinates, optical_frame_prior) in enumerate(zip(all_image_coordinates, all_object_coordinates, optical_frame_priors)):
        if is_checkerboard:
            matched_object_coordinates.append(match_checkerboard_corners(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))
        else:
            matched_object_coordinates.append(match_markers(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))

    matched_object_coordinates = np.array(matched_object_coordinates)
    
    calibration_transforms = []
    for image_coordinates, object_coordinates, camera_rb_pose in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
        optical_frame_pose = estimate_pose_ransac(image_coordinates, object_coordinates, camera_matrix, distortion_coeffs)
        camera_link_pose = np.matmul(optical_frame_pose, np.linalg.inv(T_c_o))
        calibration_transform = np.matmul(np.linalg.inv(camera_rb_pose), camera_link_pose)
        calibration_transforms.append(calibration_transform)
        
    calibration_tfs = Ts_to_tfs(calibration_transforms)
    calibration_tf_inliers, _ = remove_outliers(calibration_tfs)
    
    initial_estimate = np.mean(calibration_tf_inliers, axis=0)
    initial_estimate = transformations.compose_matrix(translate=initial_estimate[:3], angles=np.radians(initial_estimate[3:]))
    optical_frame_priors = calculate_optical_frame_poses(camera_rb_poses, initial_estimate, T_c_o)
    matched_object_coordinates = []
    for i, (image_coordinates, object_coordinates, optical_frame_prior) in enumerate(zip(all_image_coordinates, all_object_coordinates, optical_frame_priors)):
        matched_object_coordinates.append(match_markers(image_coordinates, object_coordinates, optical_frame_prior, camera_matrix, distortion_coeffs))
    matched_object_coordinates = np.array(matched_object_coordinates)
    
    better_calibration_transforms = []
    for image_coordinates, object_coordinates, camera_rb_pose in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
        optical_frame_pose = estimate_pose(image_coordinates, object_coordinates, camera_matrix, distortion_coeffs, initial_estimate)
        camera_link_pose = np.matmul(optical_frame_pose, np.linalg.inv(T_c_o))
        calibration_transform = np.matmul(np.linalg.inv(camera_rb_pose), camera_link_pose)
        better_calibration_transforms.append(calibration_transform)

    better_calibration_tfs = Ts_to_tfs(better_calibration_transforms)
    better_calibration_tfs, mask = remove_outliers(better_calibration_tfs)
    
    better_calibration = np.mean(better_calibration_tfs, axis=0)
    better_calibration = transformations.compose_matrix(translate=better_calibration[:3], angles=np.radians(better_calibration[3:]))
    
    # Filter outlier frames
    count = len(all_image_coordinates)
    all_image_coordinates = all_image_coordinates[mask]
    matched_object_coordinates = matched_object_coordinates[mask]
    camera_rb_poses = camera_rb_poses[mask]
    print("Num inliers {}/{}".format(len(all_image_coordinates), count))

    # Project object points into camera rigid body frame beforehand, as one big array (not separated by bags)
    T_RO = np.dot(better_calibration, np.linalg.inv(T_c_o))
    object_coords_R = []

    for image_coords, object_coords_M_sorted, T_MRs in zip(all_image_coordinates, matched_object_coordinates, camera_rb_poses):
        for i in range(len(image_coords)):
            object_coords_R.append(apply_T(np.linalg.inv(T_MRs), np.array([object_coords_M_sorted[i]])))
    object_coords_R = np.vstack(object_coords_R)
    
    image_coords = np.vstack(np.vstack(all_image_coordinates))
    prior = T_to_tf(better_calibration)

    res = scipy.optimize.least_squares(f, prior, verbose=0)
    return res.x 
    
    
result = calibrate(all_image_coordinates, all_object_coordinates, T_c_o, camera_rb_poses, camera_matrix, distortion_coeffs, True)
print(result)

def calibrate_k_mean(k, file_, is_checkerboard):
    """
    Do k-mean cross validation
    """
    pass

Num inliers 12/32
[-0.01318685  0.06355566  0.02056703 -0.0513646   0.05416508 -0.05655019]
